# this should be in backend
testing here for convenience

In [4]:
import torch
import pandas as pd
import json
import os
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util

from svs import SVS
from queryOpenAI import run_engine

In [5]:
# preloading slow things
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')
f = open('../models/mimic-iii/qqdict.json')
qqdict = json.load(f)
corpus_embedding = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

client = OpenAI(api_key=os.getenv("OAI_KEY"))

In [6]:
question = "What antibiotic was prescribed to patient 1125?"

In [7]:
def backend(question):
    qqlist = [qqdict[str(i)] for i in SVS(question, k=15, corpus=corpus_embedding, model=model)]

    relevant_qtext = "# Q: \t" + "\n# Q: \t".join(["\n# A: \t".join(rel) for rel in qqlist])
    test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

    result = run_engine(question, test_prompt, relevant_qtext)

    print(result)
    print()
    return result
    
backend(question)

# Q: 	tell me the name of the drug that patient 10431 was prescribed with during the same hospital visit after being diagnosed until 05/2101 with neutropenia nos?
# A: 	select t2.drug from ( select admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id from diagnoses_icd join admissions on diagnoses_icd.hadm_id = admissions.hadm_id where admissions.subject_id = 10431 and diagnoses_icd.icd9_code = ( select d_icd_diagnoses.icd9_code from d_icd_diagnoses where d_icd_diagnoses.short_title = 'neutropenia nos' ) and strftime('%y-%m',diagnoses_icd.charttime) <= '2101-05' ) as t1 join ( select admissions.subject_id, prescriptions.drug, prescriptions.startdate, admissions.hadm_id from prescriptions join admissions on prescriptions.hadm_id = admissions.hadm_id where admissions.subject_id = 10431 and strftime('%y-%m',prescriptions.startdate) <= '2101-05' ) as t2 on t1.subject_id = t2.subject_id where t1.charttime < t2.startdate and t1.hadm_id = t2.hadm_id
# Q: 	was the furosemide pres

"SELECT DRUG \nFROM PRESCRIPTIONS \nWHERE SUBJECT_ID = 1125 \nAND DRUG LIKE '%ANTIBIOTIC%';"

In [8]:
test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

In [5]:
print(question)
print()
print("SELECT COUNT(*)>0 \nFROM PRESCRIPTIONS \nWHERE PRESCRIPTIONS.HADM_ID IN \n( \n    SELECT ICUSTAYS.HADM_ID \n    FROM ICUSTAYS \n    WHERE ICUSTAYS.SUBJECT_ID = 22973 \n    ORDER BY ICUSTAYS.INTIME DESC \n    LIMIT 1 \n) \nAND PRESCRIPTIONS.DRUG = 'INSULIN';")

Was patient 22973 prescribed insulin the last time they were at the ICU?

SELECT COUNT(*)>0 
FROM PRESCRIPTIONS 
WHERE PRESCRIPTIONS.HADM_ID IN 
( 
    SELECT ICUSTAYS.HADM_ID 
    FROM ICUSTAYS 
    WHERE ICUSTAYS.SUBJECT_ID = 22973 
    ORDER BY ICUSTAYS.INTIME DESC 
    LIMIT 1 
) 
AND PRESCRIPTIONS.DRUG = 'INSULIN';
